# Eat Safe, Love

Import the dataset with mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json

## Notebook Set Up

In [ ]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [ ]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [ ]:
# assign the uk_food database to a variable name
uk_food = mongo['uk_food']

In [ ]:
# review the collections in our database
print(uk_food.list_collection_names())

In [ ]:
# assign the collection to a variable
establishments = uk_food['establishments']

In [ ]:
findone = uk_food.establishments.find_one()
pprint(findone)

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [ ]:
# Find the establishments with a hygiene score of 20
query = {'scores.Hygiene': 20}

# Use count_documents to display the number of documents in the result
print('Establishments with a hygiene score of 20:', establishments.count_documents(query))

# Display the first document in the results using pprint
pprint("First result:")
results = establishments.find(query)
pprint(results[0])

In [ ]:
# Convert the result to a Pandas DataFrame
import pandas as pd
list_ = list(results)
establishments_df = pd.DataFrame(list_)
establishments_df

# Display the number of rows in the DataFrame
print("The number of rows is:", len(establishments_df))

# Display the first 10 rows of the DataFrame
establishments_df.head(10)

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [ ]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {'LocalAuthorityName': {'$regex':'London'},'RatingValue': {'$gte': '4'}}
results = establishments.find(query)
for x in results:
    pprint(x)

# Use count_documents to display the number of documents in the result
print('Establishments with London as the Local Authority and a RatingValue greater than or equal to 4:',\
      establishments.count_documents(query))

# Display the first document in the results using pprint
pprint("First result:")
results = establishments.find(query)
pprint(results[0])

In [ ]:
# Convert the result to a Pandas DataFrame
list2 = list(results)
londondf = pd.DataFrame(list2)
londondf

# Display the number of rows in the DataFrame
print("The number of rows is:", len(londondf))

# Display the first 10 rows of the DataFrame
londondf.head()

### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [ ]:
pprint(establishments.find_one())

In [ ]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score


# Find lat and long of desired search 
degree_search = 0.01
latitude = 51.49014200
longitude = 0.08384000

lat_max = latitude + degree_search
lat_min = latitude - degree_search
long_max = longitude + degree_search
long_min = longitude - degree_search

# Query
query = {'RatingValue': '5',
         'geocode.latitude': {'$lte': lat_max, '$gte': lat_min},
         'geocode.longitude': {'$lte': long_max,'$gte': long_min}}

sort = [('scores.Hygiene', 1)]
         
limit = 5
         


# Print the results
results = list(establishments.find(query).sort(sort).limit(limit))
pprint(results)

In [ ]:
# Convert result to Pandas DataFrame
latlongdf = pd.DataFrame(results)
latlongdf

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [ ]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
 match_query = {'$match': {'scores.Hygiene':0}}

# 2. Groups the matches by Local Authority
 group_query = {'$group': {'_id': '$LocalAuthorityName',
              'count': { '$sum': 1}}}

# 3. Sorts the matches from highest to lowest
 sort_values = {'$sort': {'count': -1}}

# Print the number of documents in the result
pipeline = [match_query, group_query, sort_values]
results = establishments.aggregate(pipeline)
results_list = list(results)
print("Number of rows in result: ", len(results_list))

# Print the first 10 results
tenresults = pprint(results_list[0:10])
tenresults

In [ ]:
# Convert the result to a Pandas DataFrame
 results = establishments.aggregate(pipeline)
tendf = pd.DataFrame(results)
tendf

# Display the number of rows in the DataFrame
len(tendf)

# Display the first 10 rows of the DataFrame
tendf.head(10)